In [1]:
# Setup
from scipy.io import wavfile
import numpy as np
import sounddevice as sd
import soundfile as sf
import matplotlib.pyplot as plt
from scipy import signal
import sys
import requests
import io

# URL of the raw wave file on GitHub
# un-comment the URL of the file you want to use
# ATF Drums
#github_url = "https://raw.githubusercontent.com/kevbrnen/Music-Technology-Final-Project/main/Sound%20Examples/Testing_Files/Around%20the%20fur%20drums.wav"
# KYW inst
github_url = "https://raw.githubusercontent.com/kevbrnen/Music-Technology-Final-Project/main/Sound%20Examples/Testing_Files/KYW%20Inst.wav"

# Fetch the wave file from GitHub
response = requests.get(github_url)

# Check if the request was successful
if response.status_code == 200:
    # Read the wave data from the response content
    wave_data = io.BytesIO(response.content)
    
    # Read the wave file 
    fs, data = wavfile.read(wave_data)
    
else:
    print("Failed to fetch the wave file from GitHub")


# Get the bit depth of the audio data (assumes audio_data is a NumPy array)
bit_depth = data.dtype.itemsize * 8

# Calculate the scaling factor for normalization
scaling_factor = 2 ** (bit_depth - 1)  # For signed audio

# Convert audio data to floating-point values and normalize
data = data.astype(np.float32) / scaling_factor

# Verify that the data is now in the range -1 to 1
print(f"Min Value: {np.min(data)}")
print(f"Max Value: {np.max(data)}")

Min Value: -1.0
Max Value: 0.999969482421875


In [2]:
#### Testing individual circular buffers ####

delay_ms = 103.745923 #Delay time in ms
wet_amt = 0.5 #Wet amount (0 -> 1)

# Delay time in samples for creating buffer
delaySamps = ((delay_ms/1000) * fs)

# Creating Left and Right buffers
audio_bufferL = CircularBuffer(delaySamps)
audio_bufferR = CircularBuffer(delaySamps+94)


In [3]:
# Audio output array
audio_out = np.zeros((len(data) + max(audio_bufferL.buffer_len(), audio_bufferR.buffer_len()), 2))

# Processing Loop
for i in range(len(audio_out)):
    # Take next wet sample from buffer
    delayedL = audio_bufferL.get_sample_from_buffer()
    delayedR = audio_bufferR.get_sample_from_buffer()
    
    # Push next dry sample to buffer (if there are dry samples left)
    if(i < len(data)):
        audio_bufferL.push_sample_to_buffer(data[i, 0])
        audio_bufferR.push_sample_to_buffer(data[i, 1])

    # Combine with dry signal and put in output array
    audio_out[i, 0] = (wet_amt * delayedL) 
    audio_out[i, 1] = (wet_amt * delayedR) 
    # Include dry if there is dry left
    if(i < len(data)):
        audio_out[i, 0] += ((1 - wet_amt) * data[i, 0])
        audio_out[i, 1] += ((1 - wet_amt) * data[i, 1])



In [4]:
sd.play(audio_out, fs)

In [2]:
#### Testing Stereo Delay ####

delay_ms = 103.745923 #Delay time in ms
wet_amt = 0.4 #Wet amount (0 -> 1)

# Delay time in samples for creating buffer
delaySamps = ((delay_ms/1000) * fs)

# Stereo buffer setup with differing delay amounts for each side
stereoBuffer = StereoDelay(delaySamps, delaySamps+100)

In [3]:
# Audio output array
stereo_audio_out = np.zeros((len(data) + stereoBuffer.buffer_maxLen(), 2))

# Processing Loop
for i in range(len(stereo_audio_out)):
    # Take next wet sample from buffer
    delayed_s = stereoBuffer.get_samples_from_buffer()
    
    # Push next dry sample to buffer (if there are dry samples left)
    if(i < len(data)):
        stereoBuffer.push_samples_to_buffer(data[i])

    # Combine with dry signal and put in output array
    stereo_audio_out[i] = np.multiply(wet_amt,delayed_s)

    # Include dry if there is dry left
    if(i < len(data)):
        stereo_audio_out[i] += np.multiply((1 - wet_amt), data[i])


In [4]:
sd.play(stereo_audio_out, fs)